In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

In [2]:
# Load in the sample data
vaccine_county = pd.read_csv("Clean_data/vaccine_us_county_clean.csv",low_memory=False)
vaccine_county_sample = pd.read_csv("Clean_data/vaccine_county_sample.csv",low_memory=False)

In [3]:
vaccine_rate = vaccine_county.Series_Complete_Pop_Pct
vaccine_rate_sample = vaccine_county_sample.Series_Complete_Pop_Pct
print(vaccine_rate)
print(vaccine_rate_sample)

0       47.1
1       34.4
2       40.4
3       50.1
4       52.6
        ... 
2949    49.4
2950    31.7
2951    55.0
2952    42.5
2953    36.7
Name: Series_Complete_Pop_Pct, Length: 2954, dtype: float64
0      45.4
1      34.2
2      19.4
3      31.7
4      28.3
       ... 
995    49.4
996    52.2
997    42.1
998    33.1
999    37.8
Name: Series_Complete_Pop_Pct, Length: 1000, dtype: float64


In [ ]:
# Please use the space below for running code for cleaning your data set and saving output in the Clean_Data directory

In [ ]:
# Eugene's codes start here


In [ ]:
# Eugene's codes end here

In [ ]:
# Feipeng's codes start here

In [ ]:
# Feipeng's codes end here

In [ ]:
# Ricardo's codes start here

In [ ]:
# Ricardo's codes end here

In [ ]:
# Jenny's codes start here

In [ ]:
# Jenny's codes end here